# Saving and Restoring TensorFlow Models

### © Jubeen Shah 2018

Hey there! Welcome to `J.S Codes` jupyter notebooks for TensorFlow!
<br>
In this notebook we'll see `Saving and Restoring TensorFlow Models`.

Training a model can take hours. But once you close your TensorFlow session, you lose all the trained weights and biases. If you were to reuse the model in the future, you would have to train it all over again!

Fortunately, TensorFlow gives you the ability to save your progress using a class called `tf.train.Saver`. This class provides the functionality to save any `tf.Variable` to your file system.

In [1]:
import tensorflow as tf

In [2]:
save_file = './model.ckpt'

In [3]:
weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([3]))

saver = tf.train.Saver()

In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Weights :" )
    print(sess.run(weights))
    print("Bias :")
    print(sess.run(bias))
          
    saver.save(sess, save_path=save_file)

Weights :
[[ 0.46332362 -1.1260914   1.0018638 ]
 [ 0.8683329   0.4443242  -0.17865662]]
Bias :
[ 0.36229575 -0.7549259  -1.002118  ]


The Tensors `weights` and `bias` are set to random values using the `tf.truncated_normal()` function. The values are then saved to the save_file location, `"model.ckpt"`, using the `tf.train.Saver.save()` function. (The ".ckpt" extension stands for "checkpoint".)

### Loading the Variables
Let's load the tensor variables back, into a new model

In [5]:
tf.reset_default_graph()

In [6]:
weights = tf.Variable(tf.truncated_normal([2,3]))
bias = tf.Variable(tf.truncated_normal([3]))


In [7]:
saver = tf.train.Saver()

In [8]:
with tf.Session() as sess:
    saver.restore(sess, save_file)
    print("Weights : ")
    print(sess.run(weights))
    print("Bias : ")
    print(sess.run(bias))

INFO:tensorflow:Restoring parameters from ./model.ckpt
Weights : 
[[ 0.46332362 -1.1260914   1.0018638 ]
 [ 0.8683329   0.4443242  -0.17865662]]
Bias : 
[ 0.36229575 -0.7549259  -1.002118  ]


In [9]:
tf.reset_default_graph()

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [12]:
learning_rate = 0.001
n_input = 784
n_classes = 10

In [13]:
mnist = input_data.read_data_sets(".", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting .\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting .\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting .\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:
features = tf.placeholder(tf.float32,shape=[None, n_input])
labels = tf.placeholder(tf.float32, shape=[None, n_classes])

In [15]:
weights = tf.Variable(tf.random_normal(shape=[n_input, n_classes]))
bias = tf.Variable(tf.random_normal(shape=[n_classes]))

In [16]:
logits = tf.add(tf.matmul(features, weights), bias)

In [17]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [18]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Let's train the model, and then save the weights

In [19]:
import math

In [20]:
save_file = './train_model_mnist.ckpt'
batch_size = 128 #reduce the batch size, if enough memory isn't available
n_epochs = 100

saver = tf.train.Saver()

In [21]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epochs in range(n_epochs):
        total_batch = math.ceil(mnist.train.num_examples / batch_size)
        
        for i in range(total_batch):
            batch_features, batch_labels = mnist.train.next_batch(batch_size=batch_size)
            sess.run(optimizer, feed_dict= {features : batch_features, labels : batch_labels})
            
        if epochs % 10 == 0:
            valid_accuracy = sess.run(accuracy, feed_dict= {features : mnist.validation.images, labels : mnist.validation.labels})
            print('Epoch {:<3} - Validation Accuracy : {}'.format(epochs, valid_accuracy))
    saver.save(sess=sess, save_path=save_file)
    print('training model is now saved!!')

Epoch 0   - Validation Accuracy : 0.06459999829530716
Epoch 10  - Validation Accuracy : 0.2425999939441681
Epoch 20  - Validation Accuracy : 0.3856000006198883
Epoch 30  - Validation Accuracy : 0.4875999987125397
Epoch 40  - Validation Accuracy : 0.5550000071525574
Epoch 50  - Validation Accuracy : 0.602400004863739
Epoch 60  - Validation Accuracy : 0.63919997215271
Epoch 70  - Validation Accuracy : 0.6682000160217285
Epoch 80  - Validation Accuracy : 0.6890000104904175
Epoch 90  - Validation Accuracy : 0.7077999711036682
training model is now saved!!


In [22]:
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, save_file)
    
    test_accuracy = sess.run(accuracy, feed_dict = {features : mnist.test.images, labels : mnist.test.labels})
    
print('Test Accuracy = {}'.format(test_accuracy))

INFO:tensorflow:Restoring parameters from ./train_model_mnist.ckpt
Test Accuracy = 0.7218000292778015
